In [37]:
#Questions:
""" GERON
Note that since instances are picked randomly, some instances may be picked several times per epoch
while others may not be picked at all. If you want to be sure that the algorithm goes through every instance
at each epoch, another approach is to shuffle the training set, then go through it instance by instance, then
shuffle it again, and so on. However, this generally converges more slowly.

""";

#mix? sgd for a while, then gd?
#include newton raphson too?

In [38]:
#ToDo:
# make more lr and init methods

In [3]:
#importing packages
import pandas as pd
import os
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score
from sklearn.linear_model import SGDClassifier

In [18]:
def ReadData():
    #importing data set(s)
    filename = 'default of credit card clients.xls'
    nanDict = {} #this does nothing with this data set
    #read file
    df = pd.read_excel(filename,header=1,skiprows=0,index_col=0,na_values=nanDict) 
    #rename last column
    df.rename(index=str, columns={"default payment next month": "defaultPaymentNextMonth"}, inplace=True)
    #Replace nonsensical values in PAY_i columns with 0
    for i in [0,2,3,4,5,6]:
        col = 'PAY_{}'.format(i)
        df[col].replace(to_replace=-2, value = 0, inplace=True)
    #shuffle dataset by row
    df.sample(frac=1)
    
    # Define features and targets 
    X = df.loc[:, df.columns != 'defaultPaymentNextMonth'].values
    y = df.loc[:, df.columns == 'defaultPaymentNextMonth'].values
    
    # Categorical variables to one-hots, setting nonsensical values to 0
    onehotencoder1 = OneHotEncoder(categories='auto')
    onehotencoder2 = OneHotEncoder(categories='auto',drop='first')

    # sets number of elements in onehot vectors automatically from data.
    Xt= ColumnTransformer(
        [("one", onehotencoder1, [1]),("two", onehotencoder2, [2,3]),],
        remainder="passthrough"
    ).fit_transform(X)

    # Train-test split
    trainingShare = 0.8
    seed  = 1
    XTrain, XTest, yTrain, yTest=train_test_split(Xt, y, train_size=trainingShare, \
                                                  test_size = 1-trainingShare,
                                                 random_state=seed)
    
    #scale data, except one-hotted
    sc = StandardScaler()
    XTrain_fitting = XTrain[:,11:]
    XTest_fitting = XTest[:,11:]
    #removes mean, scales by std
    XTrain_scaler = sc.fit_transform(XTrain_fitting)
    XTest_scaler = sc.transform(XTest_fitting)
    #puts together the complete model matrix again
    XTrain_scaled=np.c_[XTrain[:,:11],XTrain_scaler]
    XTest_scaled = np.c_[XTest[:,:11],XTest_scaler]


    
    
    return XTrain_scaled,XTest_scaled,yTrain,yTest

In [10]:
def sigmoid(s): # can be shortened to: return 1./(1.+np.exp(-s))
    out = np.zeros(s.shape)
    for i,S in enumerate(s):
        out[i]= 1./(1.+np.exp(-S))
    return out

In [11]:
def Gradient(X,theta,y):
    return (X.T@(sigmoid(X@theta)-y))

In [12]:
def CostFunction(X,theta,y):
    y_pred= sigmoid(X@theta)
    y_pred_neg = sigmoid(-X@theta)
    cost__ = -y*np.log(y_pred)-(1-y)*np.log(y_pred_neg)
    cost_ = np.sum(cost__,axis=0)
#     print(cost__)
    return cost_/len(y)

In [13]:
"""
Takes two arrays of equal length and calculates the "Accuracy score"
"""
def Accuracy(t,y):
    assert len(t)==len(y), "y and y_pred dimensions do not match ({},{}).".format(len(t),len(y))
    s = [1 if i else 0 for i in np.equal(t,y)]
    return np.sum(s)/len(t)

In [14]:
"""
this function is pretty much stolen from
https://stackoverflow.com/questions/38157972/how-to-implement-mini-batch-gradient-descent-in-python
users Ash and dsachar
"""
def BatchIterator(X,y,batch_size,shuffle=True):
    assert X.shape[0]==y.shape[0], "X and y dimensions do not match ({},{}).".format(X.shape[0],y.shape[0])
    if shuffle:
        indices = np.arange(X.shape[0])
        np.random.shuffle(indices)
    for start_index in range(0,X.shape[0],batch_size):
        end_index = min(start_index+batch_size,X.shape[0])
        if shuffle:
            batch_indices = indices[start_index:end_index]
        else:
            batch_indices = slice(start_index,end_index)
        yield X[batch_indices],y[batch_indices]

In [15]:
def simulated_annealing(t):# simulated annealing?
        t0,t1 = 5,50
        return t0/(t+t1)

In [66]:
def SGD(XTrain,yTrain,XTest,yTest,gd_method='GD',lr_method='const',init_method='rand',tol=1e-5,
        max_iters=200,eta=0.00001,N_epochs=200,N_batches=100,Shuffle=True,seed=1234):
    #INIT METHODS
    np.random.seed(seed)
    theta = np.random.randn(XTrain.shape[1],1) #if function is convex, start point wont matter, and this is fine
    
    #LR METHODS
    #     u,s,v = np.linalg.svd(XTrain_scaled.T@XTrain_scaled)
    #     eta = 2/np.max(s) # learning rate must be smaller than this in order to converge!  
    #evt svd igjen for max gamma

    ce_prev = 1e6
    ce=1e6+1
    
    #GD METHODS
    if gd_method=='GD':
        c = 0
        while abs(ce-ce_prev)>tol and c<max_iters:
            ce_prev=ce
            gradient = Gradient(XTrain,theta,yTrain)
            theta -= eta*gradient*0.9999
            ce = CostFunction(XTrain,theta,yTrain) #used for convergence check
            c+=1
    if gd_method=='SGD':
        for epoch in range(N_epochs):
            for i in range(XTrain.shape[0]):
                random_index = np.random.randint(XTrain.shape[0])
                xi = XTrain[random_index:random_index+1]
                yi = yTrain[random_index:random_index+1]
                gradients = Gradient(xi,theta,yi)
#                 eta=simulated_annealing(epoch*N_batches+i) bad with or withoud this
                theta -=eta*gradients
    if gd_method =='SGDmb':
        for i in range(N_epochs):
            for batch in BatchIterator(XTrain,yTrain,int(XTrain.shape[0]/N_batches)+1,shuffle=Shuffle):
                X_batch,y_batch = batch
                gradients = Gradient(X_batch,theta,y_batch)
                theta -= eta*gradients
    if gd_method == 'skl':
        sgd_clf = SGDClassifier(random_state=1,loss='log',tol=1e-5,max_iter=500,penalty='none',alpha=0.,eta0=eta,learning_rate='constant',fit_intercept=False)
        sgd_clf.fit(XTrain,yTrain.ravel());
        theta = sgd_clf.coef_[0,:].reshape(-1,1)
        print(sgd_clf.coef_)
    #CALC AND OUTPUT ACCURACY
    y_pred_ = sigmoid(XTest@theta)
    y_pred_[y_pred_>=0.5]=1
    y_pred_[y_pred_<0.5]=0
    
    strings = ["Accuracy baseline: ","Accuracy of {} method: ".format(method)]
    print("{:26s}   {:5f}".format(strings[0],Accuracy(yTest,np.zeros((yTest.shape)))))
    print("{:26s}   {:5f}".format(strings[1],Accuracy(yTest,y_pred_)))

In [60]:
#Read data and define methods and initialization seeds to run over
XTrain,XTest,yTrain,yTest = ReadData()
methods = ['GD','SGD','SGDmb','skl']
seeds = [76,9865,75,34,875]

In [61]:
#run regressions
for seed in seeds:
    print("Seed is {}.".format(seed))
    for method in methods:
        SGD(XTrain,yTrain,XTest,yTest,gd_method=method,seed=seed)

Seed is 76.
Accuracy baseline:           0.777167
Accuracy of GD method:       0.811000
Accuracy baseline:           0.777167
Accuracy of SGD method:      0.810833
Accuracy baseline:           0.777167
Accuracy of SGDmb method:    0.810833
Accuracy baseline:           0.777167
Accuracy of skl method:      0.808500
Seed is 9865.
Accuracy baseline:           0.777167
Accuracy of GD method:       0.806500
Accuracy baseline:           0.777167
Accuracy of SGD method:      0.807333
Accuracy baseline:           0.777167
Accuracy of SGDmb method:    0.806500
Accuracy baseline:           0.777167
Accuracy of skl method:      0.808500
Seed is 75.
Accuracy baseline:           0.777167
Accuracy of GD method:       0.809667
Accuracy baseline:           0.777167
Accuracy of SGD method:      0.810000
Accuracy baseline:           0.777167
Accuracy of SGDmb method:    0.809667
Accuracy baseline:           0.777167
Accuracy of skl method:      0.808500
Seed is 34.
Accuracy baseline:           0.777167
